#Boston 311 v7 - Creating Unit Tests For Our Data Cleaning Functions

In the last 3 notebooks, we have received different results from our linear regression and logistic regression models depending on the data we gave it. With the addition of scenarios to our data cleaning functions, and the recent refactoring our of data cleaning functions, we can't be certain that there aren't some nonobvious bugs that have been introduced in the code and are impacting our machine learning models.

We therefore have come to the point where writing unit tests is not optional. Ideally we would have been writing unit tests from the beginning, but hopefully you can forgive me for putting that off for a bit in my first project. I was trying to let the demands of a machine learning project reveal themselves organically, and it didn't take too long for these doubts in the accuracy of my own data to drive unit tests to the top of my priorities.

Below is our open questions and to-dos consolidated from the last notebook. Moving forward we will probably keep this list at the top of each notebook.

##Questions and To-Dos:

2. Add more features
3. clean up the data by removing outliers
6. look at the currently available android app and see what values are available to the user to select, and which categories might be assigned by the 311 agents after receiving a new case.
7. compare a basic model which only uses the department value as a feature to our more complex models as a heuristic for whether additional features actually improve predictions.
8. Moving forward compare our model predictions with the target date assigned by 311 to see which performs better.

Questions to answer:
1. Can we find some basic commonality between open cases?
2. When and how is the target date set? How about the overdue flag?
3. Do cases autoclose after a certain time?

##Lessons from this notebook

I ran this code and obtained these results further down in this notebook:

```
df_all['type'].nunique()

result: 212
```

This output tells us that there are 212 unique results for the type column, which is a lot, but not that many for our 2 million cases. This might be a defining and predictive categorical variable worth including as a feature.

```
data = df_all.copy()
data['open_dt'] = pd.to_datetime(data['open_dt'])
data['closed_dt'] = pd.to_datetime(data['closed_dt'])
```
```
data.loc[data['closed_dt'] < data['open_dt']]['source'].value_counts()

result: 
Employee Generated    437
City Worker App       353
Constituent Call        1
Name: source, dtype: int64
```

This output tells us first of all that all but one case with a negative survival time was generated by a city worker or an employee. I had already been considering removing any cases that were employee generated. From some ad hoc case inspection, it looks like the public works department uses this database to track filling in potholes even if they haven't been reported by a constituent. That information would probably not be relevant to a model that's supposed to predict how long it takes to address constituent concerns. 

Lastly, it might be worth considering dropping cases that are closed too soon after being opened, since our model might be most useful for predicting how long it takes to close a case that doesn't have a quick resolution.

These considerations can lead to new scenarios for model training to compare to previous ones.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import glob
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime

from IPython.display import display

%matplotlib inline

#first of course we must import the necessary modules

Let's add some code to load the records directly from their storage so we don't have to upload them to google colaboratory each time. So far the URLs appear to be constant.

In [3]:
url_2023 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp9g_820k8.csv"
url_2022 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmph4izx_fb.csv"
url_2021 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmppgq9965_.csv"
url_2020 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv"
url_2019 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv"
url_2018 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv"
url_2017 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv"
url_2016 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv"
url_2015 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv"
url_2014 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/bdae89c8-d4ce-40e9-a6e1-a5203953a2e0/download/311_service_requests_2014.csv"
url_2013 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/407c5cd0-f764-4a41-adf8-054ff535049e/download/311_service_requests_2013.csv"
url_2012 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/382e10d9-1864-40ba-bef6-4eea3c75463c/download/311_service_requests_2012.csv"
url_2011 = "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/94b499d9-712a-4d2a-b790-7ceec5c9c4b1/download/311_service_requests_2011.csv"


Let's refactor the code to make fewer copies of the dataframe. Let's also add "scenario" functionality so we can keep code cleaning procedures we tried in previous notebooks. 

In [41]:
def clean_and_split_for_logistic(myData, scenario) :

  data = myData.copy()
  # Convert the 'open_dt' and 'close_dt' columns to datetime
  data['open_dt'] = pd.to_datetime(data['open_dt'])
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
  data['survival_time'] = data['closed_dt'] - data['open_dt']
  data['event'] = data['closed_dt'].notnull().astype(int)
  data['ward_number'] = data['ward'].str.extract(r'0*(\d+)')

  

  cols_to_keep = ['case_enquiry_id', 'survival_time', 'event', 'subject', 'reason', 'department', 'source', 'ward_number']
  cols_to_drop = [
 'open_dt',
 'target_dt',
 'closed_dt',
 'ontime',
 'case_status',
 'closure_reason',
 'case_title',
 'type',
 'queue',
 'submittedphoto',
 'closedphoto',
 'location',
 'fire_district',
 'pwd_district',
 'city_council_district',
 'police_district',
 'neighborhood',
 'neighborhood_services_district',
 'ward',
 'precinct',
 'location_street_name',
 'location_zipcode',
 'latitude',
 'longitude']

  #scenarios
  #scenario 0: no outlier adjustments
  
  #scenario 1: drop any open cases from the last month, and switch the event value for any cases that took longer than a month to close
  if (scenario == 1) :
    # Convert the date string to a pandas Timestamp object
    cutoff_date = pd.Timestamp('2023-04-09')

    # Filter the DataFrame to include only rows where event is 1 or open_dt is before the cutoff date
    data = data[(data['event'] == 1) | (data['open_dt'] < cutoff_date)]

    #switch the event value for any cases that took longer than a month to close
    delta = pd.Timedelta(seconds=2678400)
    data.loc[(data['event'] == 1) & (data['survival_time'] > delta), 'event'] = 0


  data = data.drop(cols_to_drop, axis=1)

  data = pd.get_dummies(data, columns=['subject', 'reason', 'department', 'source', 'ward_number'])



  #fix this line to also drop the case_enquiry_id
  X = data.drop(['case_enquiry_id','event', 'survival_time'], axis=1)
  y = data['event']

  return X, y

def clean_and_split_for_linear(myData, scenario) :

  data = myData.copy()
  # Convert the 'open_dt' and 'close_dt' columns to datetime
  data['open_dt'] = pd.to_datetime(data['open_dt'])
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
  data['survival_time'] = data['closed_dt'] - data['open_dt']
  data['event'] = data['closed_dt'].notnull().astype(int)
  data['ward_number'] = data['ward'].str.extract(r'0*(\d+)')

  cols_to_keep = ['case_enquiry_id', 'survival_time', 'event', 'subject', 'reason', 'department', 'source', 'ward_number']
  cols_to_drop = [
 'open_dt',
 'target_dt',
 'closed_dt',
 'ontime',
 'case_status',
 'closure_reason',
 'case_title',
 'type',
 'queue',
 'submittedphoto',
 'closedphoto',
 'location',
 'fire_district',
 'pwd_district',
 'city_council_district',
 'police_district',
 'neighborhood',
 'neighborhood_services_district',
 'ward',
 'precinct',
 'location_street_name',
 'location_zipcode',
 'latitude',
 'longitude']

  data = data.drop(cols_to_drop, axis=1)

  data = pd.get_dummies(data, columns=['subject', 'reason', 'department', 'source', 'ward_number'])
  data_survival_mask = data["survival_time"].notnull()
  clean_data = data[data_survival_mask].copy()
  clean_data['survival_time_hours'] = clean_data['survival_time'].apply(lambda x: x.total_seconds()/3600)

  #add scenarios
  #scenario 0: no outlier adjustments

  #scenario 1: remove records if the case took more than a month to close or time to close is negative
  if (scenario == 1) :
    clean_data = clean_data[(clean_data['survival_time_hours'] >= 0) & (clean_data['survival_time_hours'] <= 744)]
  
  #scenario 2: remove records just if the time to close is negative 
  if (scenario == 2) :
    clean_data = clean_data[(clean_data['survival_time_hours'] >= 0)]

  #fix this line to also drop the case_enquiry_id
  X = clean_data.drop(['case_enquiry_id','survival_time_hours', 'survival_time', 'event'], axis=1) 
  y = clean_data['survival_time_hours']
  
  return X, y

Let's add some unit tests. The easiest way to create unit tests here is to pull some records from our data, and modify a few so our data covers all of the scenarios in our data cleaning functions. Let's outline the scenarios here:

##Logistic cleaning:
###Scenario 0: 
No outlier removal. y output is a series of 0 or 1 corresponding to whether a case is Open or Closed, with 0 marking Open and 1 marking closed. X dataframe should contain only dummied columns for the 'subject', 'reason', 'department', 'source', and 'ward_number' columns.
###Scenario 1: 
drop any open cases from the last month, and switch the event value for any cases that took longer than a month to close. Otherwise the same as Scenario 0.

##Linear cleaning:
###Scenario 0: 
No outlier removal. y output is a series of floats corresponding to the number of hours between case open date and case close date. All open cases are dropped.
###Scenario 1:
remove records if the case took more than a month to close or time to close is negative
###scenario 2: 
remove records only if the time to close is negative

##How to add a new scenario and create a new test
When we add scenarios, we will want to check it didn't break other scenarios add a new test. To do that, we will:

1. Add the scenario code to the data cleaning function.
2. Run the current unit test function.
3. Run the new scenario on the test sample data.
4. Inspect the output visually to see if the scenario is working. 
5. Print and copy the verified output into our testing function.
6. Add the scenario call and assertions to the testing function.

##Pull data and print it
We have to pull data, modify it to encompass all scenarios, and then inspect the current output for errors. Once we establish the functions currently work as expected, we can save hardcoded copies of the data to create unit tests to run after future changes to the functions.

In [72]:
df2022 = pd.read_csv(url_2022);

In [73]:
import pprint

pprint.pprint(df2022.head(10).to_dict(orient='list'))

{'case_enquiry_id': [101004125189,
                     101004161747,
                     101004149944,
                     101004113302,
                     101004122704,
                     101004122479,
                     101004113310,
                     101004113311,
                     101004113328,
                     101004113550],
 'case_status': ['Open',
                 'Closed',
                 'Open',
                 'Closed',
                 'Open',
                 'Open',
                 'Closed',
                 'Closed',
                 'Open',
                 'Closed'],
 'case_title': ['Illegal Rooming House',
                'PublicWorks: Complaint',
                'Space Savers',
                'Parking Enforcement',
                'DISPATCHED Heat - Excessive  Insufficient',
                'Generic Noise Disturbance',
                'Parking Enforcement',
                'General Lighting Request',
                'Loud Parties/Music/People',


In [74]:
test_data_2022 = pd.DataFrame({'case_enquiry_id': [101004125189,
                     101004161747,
                     101004149944,
                     101004113302,
                     101004122704,
                     101004122479,
                     101004113310,
                     101004113311,
                     101004113328,
                     101004113550],
 'case_status': ['Open',
                 'Closed',
                 'Open',
                 'Closed',
                 'Open',
                 'Open',
                 'Closed',
                 'Closed',
                 'Open',
                 'Closed'],
 'case_title': ['Illegal Rooming House',
                'PublicWorks: Complaint',
                'Space Savers',
                'Parking Enforcement',
                'DISPATCHED Heat - Excessive  Insufficient',
                'Generic Noise Disturbance',
                'Parking Enforcement',
                'General Lighting Request',
                'Loud Parties/Music/People',
                'Requests for Street Cleaning'],
 'city_council_district': ['4', ' ', '4', '2', '7', '8', '3', '6', '1', '2'],
 'closed_dt': [np.nan,
               '2021-02-02 11:45:47',
               np.nan,
               '2022-01-03 00:13:17',
               np.nan,
               np.nan,
               '2022-01-03 00:13:02',
               '2022-04-02 13:01:14',
               np.nan,
               '2022-05-03 05:59:20'],
 'closedphoto': [np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 'https://spot-boston-res.cloudinary.com/image/upload/v1641207557/boston/production/o0vkrv9zckukp8httr7g.jpg'],
 'closure_reason': [' ',
                    'Case Closed Case Noted    ',
                    ' ',
                    'Case Closed. Closed date : 2022-01-03 00:13:17.393 Case '
                    'Resolved CLEAR ',
                    ' ',
                    ' ',
                    'Case Closed. Closed date : 2022-01-03 00:13:02.72 Case '
                    'Resolved CLEAR ',
                    'Case Closed. Closed date : Sat Apr 02 13:01:14 EDT 2022 '
                    'Noted ',
                    ' ',
                    'Case Closed. Closed date : Mon Jan 03 05:59:20 EST 2022 '
                    'Noted 3 bags of trash collected at intersection of '
                    'Dartmouth and Warren at 5:56 a.m. on Monday 1/3/22. We '
                    'will return on next scheduled trash day. '],
 'department': ['ISD',
                'PWDx',
                'PWDx',
                'BTDT',
                'ISD',
                'INFO',
                'BTDT',
                'PWDx',
                'INFO',
                'PWDx'],
 'fire_district': ['8', ' ', '8', '6', '9', '3', '8', '9', '3', '4'],
 'latitude': [42.2896,
              42.3594,
              42.2876,
              42.3594,
              42.311,
              42.3657,
              42.291,
              42.3594,
              42.3669,
              42.3594],
 'location': ['27 Lithgow St  Dorchester  MA  02124',
              ' ',
              '492 Harvard St  Dorchester  MA  02124',
              'INTERSECTION of Seaport Blvd & Sleeper St  Boston  MA  ',
              '15 Crawford St  Dorchester  MA  02121',
              '50-150 Causeway St  Boston  MA  02114',
              '16 Frost Ave  Dorchester  MA  02122',
              'INTERSECTION of Boylston St & Moraine St  Jamaica Plain  MA  ',
              '194 Salem St  Boston  MA  02113',
              'INTERSECTION of Warren Ave & Dartmouth St  Boston  MA  '],
 'location_street_name': ['27 Lithgow St',
                          np.nan,
                          '492 Harvard St',
                          'INTERSECTION Seaport Blvd & Sleeper St',
                          '15 Crawford St',
                          '50-150 Causeway St',
                          '16 Frost Ave',
                          'INTERSECTION Boylston St & Moraine St',
                          '194 Salem St',
                          'INTERSECTION Warren Ave & Dartmouth St'],
 'location_zipcode': [2124.0,
                      np.nan,
                      2124.0,
                      np.nan,
                      2121.0,
                      2114.0,
                      2122.0,
                      np.nan,
                      2113.0,
                      np.nan],
 'longitude': [-71.0701,
               -71.0587,
               -71.0936,
               -71.0587,
               -71.0841,
               -71.0617,
               -71.0503,
               -71.0587,
               -71.0546,
               -71.0587],
 'neighborhood': ['Dorchester',
                  ' ',
                  'Greater Mattapan',
                  'South Boston / South Boston Waterfront',
                  'Roxbury',
                  'Boston',
                  'Dorchester',
                  'Jamaica Plain',
                  'Downtown / Financial District',
                  'South End'],
 'neighborhood_services_district': ['8',
                                    ' ',
                                    '9',
                                    '5',
                                    '13',
                                    '3',
                                    '7',
                                    '11',
                                    '3',
                                    '6'],
 'ontime': ['OVERDUE',
            'ONTIME',
            'ONTIME',
            'ONTIME',
            'OVERDUE',
            'ONTIME',
            'ONTIME',
            'OVERDUE',
            'ONTIME',
            'ONTIME'],
 'open_dt': ['2023-05-09 12:59:00',
             '2022-02-02 11:42:49',
             '2022-01-28 19:36:00',
             '2022-01-01 00:36:24',
             '2022-01-11 09:47:00',
             '2022-01-10 21:49:00',
             '2022-01-01 01:13:52',
             '2022-01-01 01:14:39',
             '2022-01-01 03:08:00',
             '2022-01-01 13:51:00'],
 'police_district': ['C11',
                     ' ',
                     'B3',
                     'C6',
                     'B2',
                     'A1',
                     'C11',
                     'E13',
                     'A1',
                     'D4'],
 'precinct': ['1706',
              ' ',
              '1411',
              '0601',
              '1202',
              ' ',
              '1607',
              '1903',
              '0302',
              '0401'],
 'pwd_district': ['07', ' ', '07', '05', '10B', '1B', '07', '02', '1B', '1C'],
 'queue': ['ISD_Housing (INTERNAL)',
           'PWDx_General Comments',
           'PWDx_Space Saver Removal',
           'BTDT_Parking Enforcement',
           'ISD_Housing (INTERNAL)',
           'INFO01_GenericeFormforOtherServiceRequestTypes',
           'BTDT_Parking Enforcement',
           'PWDx_Street Light_General Lighting Request',
           'INFO01_GenericeFormforOtherServiceRequestTypes',
           'PWDx_Missed Trash\\Recycling\\Yard Waste\\Bulk Item'],
 'reason': ['Building',
            'Employee & General Comments',
            'Sanitation',
            'Enforcement & Abandoned Vehicles',
            'Housing',
            'Generic Noise Disturbance',
            'Enforcement & Abandoned Vehicles',
            'Street Lights',
            'Noise Disturbance',
            'Street Cleaning'],
 'source': ['Constituent Call',
            'Constituent Call',
            'Constituent Call',
            'Citizens Connect App',
            'Constituent Call',
            'Constituent Call',
            'Citizens Connect App',
            'City Worker App',
            'Constituent Call',
            'Citizens Connect App'],
 'subject': ['Inspectional Services',
             "Mayor's 24 Hour Hotline",
             'Public Works Department',
             'Transportation - Traffic Division',
             'Inspectional Services',
             "Mayor's 24 Hour Hotline",
             'Transportation - Traffic Division',
             'Public Works Department',
             'Boston Police Department',
             'Public Works Department'],
 'submittedphoto': [np.nan,
                    np.nan,
                    np.nan,
                    'https://311.boston.gov/media/boston/report/photos/61cfe84b05bbcf180c293ece/photo_20220101_003547.jpg',
                    np.nan,
                    np.nan,
                    'https://311.boston.gov/media/boston/report/photos/61cff11805bbcf180c2944b1/report.jpg',
                    np.nan,
                    np.nan,
                    'https://311.boston.gov/media/boston/report/photos/61d0a2af05bbcf180c2993e3/report.jpg'],
 'target_dt': ['2022-01-20 12:59:39',
               '2022-02-16 11:42:49',
               np.nan,
               '2022-01-04 08:30:00',
               '2022-02-10 09:47:22',
               np.nan,
               '2022-01-04 08:30:00',
               '2022-02-15 01:14:45',
               np.nan,
               '2022-01-04 08:30:00'],
 'type': ['Illegal Rooming House',
          'General Comments For a Program or Policy',
          'Space Savers',
          'Parking Enforcement',
          'Heat - Excessive  Insufficient',
          'Undefined Noise Disturbance',
          'Parking Enforcement',
          'General Lighting Request',
          'Loud Parties/Music/People',
          'Requests for Street Cleaning'],
 'ward': ['Ward 17',
          ' ',
          'Ward 14',
          '6',
          'Ward 12',
          '03',
          'Ward 16',
          '19',
          'Ward 3',
          '4']})

##Run the data cleaning on our test data

Now we can run the test data through data cleaning and look at the outputs, checking visually if it is correct. It would be nice to write logic to check the data, but then we'd need tests for the tests, and there's a never ending issue there.

In [79]:
logistic_test_X_0, logistic_test_y_0 = clean_and_split_for_logistic(test_data_2022, 0)
logistic_test_X_1, logistic_test_y_1 = clean_and_split_for_logistic(test_data_2022, 1)

linear_test_X_0, linear_test_y_0 = clean_and_split_for_linear(test_data_2022, 0)
linear_test_X_1, linear_test_y_1 = clean_and_split_for_linear(test_data_2022, 1)
linear_test_X_2, linear_test_y_2 = clean_and_split_for_linear(test_data_2022, 2)

logistic_test_Xy_0 = logistic_test_X_0.copy()
logistic_test_Xy_1 = logistic_test_X_1.copy()
linear_test_Xy_0 = linear_test_X_0.copy()
linear_test_Xy_1 = linear_test_X_1.copy()
linear_test_Xy_2 = linear_test_X_2.copy()

logistic_test_Xy_0['event'] = logistic_test_y_0
logistic_test_Xy_1['event'] = logistic_test_y_1

linear_test_Xy_0['survival_time_hours'] = linear_test_y_0
linear_test_Xy_1['survival_time_hours'] = linear_test_y_1
linear_test_Xy_2['survival_time_hours'] = linear_test_y_2

In [52]:
test_data_2022.head(10)

,case_enquiry_id,case_status,case_title,city_council_district,closed_dt,closedphoto,closure_reason,department,fire_district,latitude,...,precinct,pwd_district,queue,reason,source,subject,submittedphoto,target_dt,type,ward
0,101004125189,Open,Illegal Rooming House,4,NaN,NaN,,ISD,8,42.2896,...,1706,07,ISD_Housing (INTERNAL),Building,Constituent Call,Inspectional Services,NaN,2022-01-20 12:59:39,Illegal Rooming House,Ward 17
1,101004161747,Closed,PublicWorks: Complaint,,2021-02-02 11:45:47,NaN,Case Closed Case Noted,PWDx,,42.3594,...,,,PWDx_General Comments,Employee & General Comments,Constituent Call,Mayor's 24 Hour Hotline,NaN,2022-02-16 11:42:49,General Comments For a Program or Policy,
2,101004149944,Open,Space Savers,4,NaN,NaN,,PWDx,8,42.2876,...,1411,07,PWDx_Space Saver Removal,Sanitation,Constituent Call,Public Works Department,NaN,NaN,Space Savers,Ward 14
3,101004113302,Closed,Parking Enforcement,2,2022-01-03 00:13:17,NaN,Case Closed. Closed date : 2022-01-03 00:13:17...,BTDT,6,42.3594,...,0601,05,BTDT_Parking Enforcement,Enforcement & Abandoned Vehicles,Citizens Connect App,Transportation - Traffic Division,https://311.boston.gov/media/boston/report/pho...,2022-01-04 08:30:00,Parking Enforcement,6
4,101004122704,Open,DISPATCHED Heat - Excessive Insufficient,7,NaN,NaN,,ISD,9,42.3110,...,1202,10B,ISD_Housing (INTERNAL),Housing,Constituent Call,Inspectional Services,NaN,2022-02-10 09:47:22,Heat - Excessive Insufficient,Ward 12
5,101004122479,Open,Generic Noise Disturbance,8,NaN,NaN,,INFO,3,42.3657,...,,1B,INFO01_GenericeFormforOtherServiceRequestTypes,Generic Noise Disturbance,Constituent Call,Mayor's 24 Hour Hotline,NaN,NaN,Undefined Noise Disturbance,03
6,101004113310,Closed,Parking Enforcement,3,2022-01-03 00:13:02,NaN,Case Closed. Closed date : 2022-01-03 00:13:02...,BTDT,8,42.2910,...,1607,07,BTDT_Parking Enforcement,Enforcement & Abandoned Vehicles,Citizens Connect App,Transportation - Traffic Division,https://311.boston.gov/media/boston/report/pho...,2022-01-04 08:30:00,Parking Enforcement,Ward 16
7,101004113311,Closed,General Lighting Request,6,2022-04-02 13:01:14,NaN,Case Closed. Closed date : Sat Apr 02 13:01:14...,PWDx,9,42.3594,...,1903,02,PWDx_Street Light_General Lighting Request,Street Lights,City Worker App,Public Works Department,NaN,2022-02-15 01:14:45,General Lighting Request,19
8,101004113328,Open,Loud Parties/Music/People,1,NaN,NaN,,INFO,3,42.3669,...,0302,1B,INFO01_GenericeFormforOtherServiceRequestTypes,Noise Disturbance,Constituent Call,Boston Police Department,NaN,NaN,Loud Parties/Music/People,Ward 3
9,101004113550,Closed,Requests for Street Cleaning,2,2022-05-03 05:59:20,https://spot-boston-res.cloudinary.com/image/u...,Case Closed. Closed date : Mon Jan 03 05:59:20...,PWDx,4,42.3594,...,0401,1C,PWDx_Missed Trash\Recycling\Yard Waste\Bulk Item,Street Cleaning,Citizens Connect App,Public Works Department,https://311.boston.gov/media/boston/report/pho...,2022-01-04 08:30:00,Requests for Street Cleaning,4


In [80]:
logistic_test_Xy_0.head(10)

,subject_Boston Police Department,subject_Inspectional Services,subject_Mayor's 24 Hour Hotline,subject_Public Works Department,subject_Transportation - Traffic Division,reason_Building,reason_Employee & General Comments,reason_Enforcement & Abandoned Vehicles,reason_Generic Noise Disturbance,reason_Housing,...,source_Constituent Call,ward_number_12,ward_number_14,ward_number_16,ward_number_17,ward_number_19,ward_number_3,ward_number_4,ward_number_6,event
0,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,1,0,0,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
6,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [81]:
logistic_test_Xy_1.head(10)

,subject_Boston Police Department,subject_Inspectional Services,subject_Mayor's 24 Hour Hotline,subject_Public Works Department,subject_Transportation - Traffic Division,reason_Employee & General Comments,reason_Enforcement & Abandoned Vehicles,reason_Generic Noise Disturbance,reason_Housing,reason_Noise Disturbance,...,source_City Worker App,source_Constituent Call,ward_number_12,ward_number_14,ward_number_16,ward_number_19,ward_number_3,ward_number_4,ward_number_6,event
1,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,1,0,0,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
6,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
7,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
8,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [82]:
linear_test_Xy_1.head(10)

,subject_Boston Police Department,subject_Inspectional Services,subject_Mayor's 24 Hour Hotline,subject_Public Works Department,subject_Transportation - Traffic Division,reason_Building,reason_Employee & General Comments,reason_Enforcement & Abandoned Vehicles,reason_Generic Noise Disturbance,reason_Housing,...,source_Constituent Call,ward_number_12,ward_number_14,ward_number_16,ward_number_17,ward_number_19,ward_number_3,ward_number_4,ward_number_6,survival_time_hours
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,47.614722
6,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,46.986111


In [83]:
linear_test_Xy_2.head()

,subject_Boston Police Department,subject_Inspectional Services,subject_Mayor's 24 Hour Hotline,subject_Public Works Department,subject_Transportation - Traffic Division,reason_Building,reason_Employee & General Comments,reason_Enforcement & Abandoned Vehicles,reason_Generic Noise Disturbance,reason_Housing,...,source_Constituent Call,ward_number_12,ward_number_14,ward_number_16,ward_number_17,ward_number_19,ward_number_3,ward_number_4,ward_number_6,survival_time_hours
3,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,47.614722
6,0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,46.986111
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2195.776389
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2920.138889


##Print out cleaned test data to hardcode into unit tests

Now that we've inspected the cleaned data for all scenarios to ensure the output is what we expected, we can print it out for hardcoding.

In [84]:
print("tlogistic_test_X_0 = pd.DataFrame(", end='')
pprint.pprint(logistic_test_X_0.to_dict(orient='list'))
print(')')

print("tlogistic_test_X_1 = pd.DataFrame(", end='')
pprint.pprint(logistic_test_X_1.to_dict(orient='list'))
print(')')

print("tlinear_test_X_0 = pd.DataFrame(", end='')
pprint.pprint(linear_test_X_0.to_dict(orient='list'))
print(')')

print("tlinear_test_X_1 = pd.DataFrame(", end='')
pprint.pprint(linear_test_X_1.to_dict(orient='list'))
print(')')

print("tlinear_test_X_2 = pd.DataFrame(", end='')
pprint.pprint(linear_test_X_2.to_dict(orient='list'))
print(')')

print("tlogistic_test_y_0 = pd.Series(", end='')
pprint.pprint(logistic_test_y_0.to_dict())
print(')')

print("tlogistic_test_y_1 = pd.Series(", end='')
pprint.pprint(logistic_test_y_1.to_dict())
print(')')

print("tlinear_test_y_0 = pd.Series(", end='')
pprint.pprint(linear_test_y_0.to_dict())
print(')')

print("tlinear_test_y_1 = pd.Series(", end='')
pprint.pprint(linear_test_y_1.to_dict())
print(')')

print("tlinear_test_y_2 = pd.Series(", end='')
pprint.pprint(linear_test_y_2.to_dict())
print(')')


tlogistic_test_X_0 = pd.DataFrame({'department_BTDT': [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 'department_INFO': [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 'department_ISD': [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'department_PWDx': [0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
 'reason_Building': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'reason_Employee & General Comments': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'reason_Enforcement & Abandoned Vehicles': [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 'reason_Generic Noise Disturbance': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 'reason_Housing': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'reason_Noise Disturbance': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'reason_Sanitation': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'reason_Street Cleaning': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 'reason_Street Lights': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'source_Citizens Connect App': [0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
 'source_City Worker App': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'source_Constituent Call': [1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
 'subject_Boston Police De

In [118]:
from pandas.testing import assert_frame_equal, assert_series_equal

def test_data_clean_functions() :
  #set up the test data
  test_data_2022 = pd.DataFrame({'case_enquiry_id': [101004125189,
                     101004161747,
                     101004149944,
                     101004113302,
                     101004122704,
                     101004122479,
                     101004113310,
                     101004113311,
                     101004113328,
                     101004113550],
 'case_status': ['Open',
                 'Closed',
                 'Open',
                 'Closed',
                 'Open',
                 'Open',
                 'Closed',
                 'Closed',
                 'Open',
                 'Closed'],
 'case_title': ['Illegal Rooming House',
                'PublicWorks: Complaint',
                'Space Savers',
                'Parking Enforcement',
                'DISPATCHED Heat - Excessive  Insufficient',
                'Generic Noise Disturbance',
                'Parking Enforcement',
                'General Lighting Request',
                'Loud Parties/Music/People',
                'Requests for Street Cleaning'],
 'city_council_district': ['4', ' ', '4', '2', '7', '8', '3', '6', '1', '2'],
 'closed_dt': [np.nan,
               '2021-02-02 11:45:47',
               np.nan,
               '2022-01-03 00:13:17',
               np.nan,
               np.nan,
               '2022-01-03 00:13:02',
               '2022-04-02 13:01:14',
               np.nan,
               '2022-05-03 05:59:20'],
 'closedphoto': [np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 np.nan,
                 'https://spot-boston-res.cloudinary.com/image/upload/v1641207557/boston/production/o0vkrv9zckukp8httr7g.jpg'],
 'closure_reason': [' ',
                    'Case Closed Case Noted    ',
                    ' ',
                    'Case Closed. Closed date : 2022-01-03 00:13:17.393 Case '
                    'Resolved CLEAR ',
                    ' ',
                    ' ',
                    'Case Closed. Closed date : 2022-01-03 00:13:02.72 Case '
                    'Resolved CLEAR ',
                    'Case Closed. Closed date : Sat Apr 02 13:01:14 EDT 2022 '
                    'Noted ',
                    ' ',
                    'Case Closed. Closed date : Mon Jan 03 05:59:20 EST 2022 '
                    'Noted 3 bags of trash collected at intersection of '
                    'Dartmouth and Warren at 5:56 a.m. on Monday 1/3/22. We '
                    'will return on next scheduled trash day. '],
 'department': ['ISD',
                'PWDx',
                'PWDx',
                'BTDT',
                'ISD',
                'INFO',
                'BTDT',
                'PWDx',
                'INFO',
                'PWDx'],
 'fire_district': ['8', ' ', '8', '6', '9', '3', '8', '9', '3', '4'],
 'latitude': [42.2896,
              42.3594,
              42.2876,
              42.3594,
              42.311,
              42.3657,
              42.291,
              42.3594,
              42.3669,
              42.3594],
 'location': ['27 Lithgow St  Dorchester  MA  02124',
              ' ',
              '492 Harvard St  Dorchester  MA  02124',
              'INTERSECTION of Seaport Blvd & Sleeper St  Boston  MA  ',
              '15 Crawford St  Dorchester  MA  02121',
              '50-150 Causeway St  Boston  MA  02114',
              '16 Frost Ave  Dorchester  MA  02122',
              'INTERSECTION of Boylston St & Moraine St  Jamaica Plain  MA  ',
              '194 Salem St  Boston  MA  02113',
              'INTERSECTION of Warren Ave & Dartmouth St  Boston  MA  '],
 'location_street_name': ['27 Lithgow St',
                          np.nan,
                          '492 Harvard St',
                          'INTERSECTION Seaport Blvd & Sleeper St',
                          '15 Crawford St',
                          '50-150 Causeway St',
                          '16 Frost Ave',
                          'INTERSECTION Boylston St & Moraine St',
                          '194 Salem St',
                          'INTERSECTION Warren Ave & Dartmouth St'],
 'location_zipcode': [2124.0,
                      np.nan,
                      2124.0,
                      np.nan,
                      2121.0,
                      2114.0,
                      2122.0,
                      np.nan,
                      2113.0,
                      np.nan],
 'longitude': [-71.0701,
               -71.0587,
               -71.0936,
               -71.0587,
               -71.0841,
               -71.0617,
               -71.0503,
               -71.0587,
               -71.0546,
               -71.0587],
 'neighborhood': ['Dorchester',
                  ' ',
                  'Greater Mattapan',
                  'South Boston / South Boston Waterfront',
                  'Roxbury',
                  'Boston',
                  'Dorchester',
                  'Jamaica Plain',
                  'Downtown / Financial District',
                  'South End'],
 'neighborhood_services_district': ['8',
                                    ' ',
                                    '9',
                                    '5',
                                    '13',
                                    '3',
                                    '7',
                                    '11',
                                    '3',
                                    '6'],
 'ontime': ['OVERDUE',
            'ONTIME',
            'ONTIME',
            'ONTIME',
            'OVERDUE',
            'ONTIME',
            'ONTIME',
            'OVERDUE',
            'ONTIME',
            'ONTIME'],
 'open_dt': ['2023-05-09 12:59:00',
             '2022-02-02 11:42:49',
             '2022-01-28 19:36:00',
             '2022-01-01 00:36:24',
             '2022-01-11 09:47:00',
             '2022-01-10 21:49:00',
             '2022-01-01 01:13:52',
             '2022-01-01 01:14:39',
             '2022-01-01 03:08:00',
             '2022-01-01 13:51:00'],
 'police_district': ['C11',
                     ' ',
                     'B3',
                     'C6',
                     'B2',
                     'A1',
                     'C11',
                     'E13',
                     'A1',
                     'D4'],
 'precinct': ['1706',
              ' ',
              '1411',
              '0601',
              '1202',
              ' ',
              '1607',
              '1903',
              '0302',
              '0401'],
 'pwd_district': ['07', ' ', '07', '05', '10B', '1B', '07', '02', '1B', '1C'],
 'queue': ['ISD_Housing (INTERNAL)',
           'PWDx_General Comments',
           'PWDx_Space Saver Removal',
           'BTDT_Parking Enforcement',
           'ISD_Housing (INTERNAL)',
           'INFO01_GenericeFormforOtherServiceRequestTypes',
           'BTDT_Parking Enforcement',
           'PWDx_Street Light_General Lighting Request',
           'INFO01_GenericeFormforOtherServiceRequestTypes',
           'PWDx_Missed Trash\\Recycling\\Yard Waste\\Bulk Item'],
 'reason': ['Building',
            'Employee & General Comments',
            'Sanitation',
            'Enforcement & Abandoned Vehicles',
            'Housing',
            'Generic Noise Disturbance',
            'Enforcement & Abandoned Vehicles',
            'Street Lights',
            'Noise Disturbance',
            'Street Cleaning'],
 'source': ['Constituent Call',
            'Constituent Call',
            'Constituent Call',
            'Citizens Connect App',
            'Constituent Call',
            'Constituent Call',
            'Citizens Connect App',
            'City Worker App',
            'Constituent Call',
            'Citizens Connect App'],
 'subject': ['Inspectional Services',
             "Mayor's 24 Hour Hotline",
             'Public Works Department',
             'Transportation - Traffic Division',
             'Inspectional Services',
             "Mayor's 24 Hour Hotline",
             'Transportation - Traffic Division',
             'Public Works Department',
             'Boston Police Department',
             'Public Works Department'],
 'submittedphoto': [np.nan,
                    np.nan,
                    np.nan,
                    'https://311.boston.gov/media/boston/report/photos/61cfe84b05bbcf180c293ece/photo_20220101_003547.jpg',
                    np.nan,
                    np.nan,
                    'https://311.boston.gov/media/boston/report/photos/61cff11805bbcf180c2944b1/report.jpg',
                    np.nan,
                    np.nan,
                    'https://311.boston.gov/media/boston/report/photos/61d0a2af05bbcf180c2993e3/report.jpg'],
 'target_dt': ['2022-01-20 12:59:39',
               '2022-02-16 11:42:49',
               np.nan,
               '2022-01-04 08:30:00',
               '2022-02-10 09:47:22',
               np.nan,
               '2022-01-04 08:30:00',
               '2022-02-15 01:14:45',
               np.nan,
               '2022-01-04 08:30:00'],
 'type': ['Illegal Rooming House',
          'General Comments For a Program or Policy',
          'Space Savers',
          'Parking Enforcement',
          'Heat - Excessive  Insufficient',
          'Undefined Noise Disturbance',
          'Parking Enforcement',
          'General Lighting Request',
          'Loud Parties/Music/People',
          'Requests for Street Cleaning'],
 'ward': ['Ward 17',
          ' ',
          'Ward 14',
          '6',
          'Ward 12',
          '03',
          'Ward 16',
          '19',
          'Ward 3',
          '4']})

  #define the expected output
  tlogistic_test_X_0 = pd.DataFrame({'department_BTDT': [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
  'department_INFO': [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
  'department_ISD': [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  'department_PWDx': [0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
  'reason_Building': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'reason_Employee & General Comments': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  'reason_Enforcement & Abandoned Vehicles': [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
  'reason_Generic Noise Disturbance': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  'reason_Housing': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  'reason_Noise Disturbance': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  'reason_Sanitation': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  'reason_Street Cleaning': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'reason_Street Lights': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  'source_Citizens Connect App': [0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
  'source_City Worker App': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  'source_Constituent Call': [1, 1, 1, 0, 1, 1, 0, 0, 1, 0],
  'subject_Boston Police Department': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  'subject_Inspectional Services': [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  "subject_Mayor's 24 Hour Hotline": [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
  'subject_Public Works Department': [0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
  'subject_Transportation - Traffic Division': [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
  'ward_number_12': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  'ward_number_14': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  'ward_number_16': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  'ward_number_17': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'ward_number_19': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  'ward_number_3': [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
  'ward_number_4': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'ward_number_6': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]}
  )
  tlogistic_test_X_1 = pd.DataFrame({'department_BTDT': [0, 0, 1, 0, 0, 1, 0, 0, 0],
  'department_INFO': [0, 0, 0, 0, 1, 0, 0, 1, 0],
  'department_ISD': [0, 0, 0, 1, 0, 0, 0, 0, 0],
  'department_PWDx': [1, 1, 0, 0, 0, 0, 1, 0, 1],
  'reason_Employee & General Comments': [1, 0, 0, 0, 0, 0, 0, 0, 0],
  'reason_Enforcement & Abandoned Vehicles': [0, 0, 1, 0, 0, 1, 0, 0, 0],
  'reason_Generic Noise Disturbance': [0, 0, 0, 0, 1, 0, 0, 0, 0],
  'reason_Housing': [0, 0, 0, 1, 0, 0, 0, 0, 0],
  'reason_Noise Disturbance': [0, 0, 0, 0, 0, 0, 0, 1, 0],
  'reason_Sanitation': [0, 1, 0, 0, 0, 0, 0, 0, 0],
  'reason_Street Cleaning': [0, 0, 0, 0, 0, 0, 0, 0, 1],
  'reason_Street Lights': [0, 0, 0, 0, 0, 0, 1, 0, 0],
  'source_Citizens Connect App': [0, 0, 1, 0, 0, 1, 0, 0, 1],
  'source_City Worker App': [0, 0, 0, 0, 0, 0, 1, 0, 0],
  'source_Constituent Call': [1, 1, 0, 1, 1, 0, 0, 1, 0],
  'subject_Boston Police Department': [0, 0, 0, 0, 0, 0, 0, 1, 0],
  'subject_Inspectional Services': [0, 0, 0, 1, 0, 0, 0, 0, 0],
  "subject_Mayor's 24 Hour Hotline": [1, 0, 0, 0, 1, 0, 0, 0, 0],
  'subject_Public Works Department': [0, 1, 0, 0, 0, 0, 1, 0, 1],
  'subject_Transportation - Traffic Division': [0, 0, 1, 0, 0, 1, 0, 0, 0],
  'ward_number_12': [0, 0, 0, 1, 0, 0, 0, 0, 0],
  'ward_number_14': [0, 1, 0, 0, 0, 0, 0, 0, 0],
  'ward_number_16': [0, 0, 0, 0, 0, 1, 0, 0, 0],
  'ward_number_19': [0, 0, 0, 0, 0, 0, 1, 0, 0],
  'ward_number_3': [0, 0, 0, 0, 1, 0, 0, 1, 0],
  'ward_number_4': [0, 0, 0, 0, 0, 0, 0, 0, 1],
  'ward_number_6': [0, 0, 1, 0, 0, 0, 0, 0, 0]}
  )
  tlinear_test_X_0 = pd.DataFrame({'department_BTDT': [0, 1, 1, 0, 0],
  'department_INFO': [0, 0, 0, 0, 0],
  'department_ISD': [0, 0, 0, 0, 0],
  'department_PWDx': [1, 0, 0, 1, 1],
  'reason_Building': [0, 0, 0, 0, 0],
  'reason_Employee & General Comments': [1, 0, 0, 0, 0],
  'reason_Enforcement & Abandoned Vehicles': [0, 1, 1, 0, 0],
  'reason_Generic Noise Disturbance': [0, 0, 0, 0, 0],
  'reason_Housing': [0, 0, 0, 0, 0],
  'reason_Noise Disturbance': [0, 0, 0, 0, 0],
  'reason_Sanitation': [0, 0, 0, 0, 0],
  'reason_Street Cleaning': [0, 0, 0, 0, 1],
  'reason_Street Lights': [0, 0, 0, 1, 0],
  'source_Citizens Connect App': [0, 1, 1, 0, 1],
  'source_City Worker App': [0, 0, 0, 1, 0],
  'source_Constituent Call': [1, 0, 0, 0, 0],
  'subject_Boston Police Department': [0, 0, 0, 0, 0],
  'subject_Inspectional Services': [0, 0, 0, 0, 0],
  "subject_Mayor's 24 Hour Hotline": [1, 0, 0, 0, 0],
  'subject_Public Works Department': [0, 0, 0, 1, 1],
  'subject_Transportation - Traffic Division': [0, 1, 1, 0, 0],
  'ward_number_12': [0, 0, 0, 0, 0],
  'ward_number_14': [0, 0, 0, 0, 0],
  'ward_number_16': [0, 0, 1, 0, 0],
  'ward_number_17': [0, 0, 0, 0, 0],
  'ward_number_19': [0, 0, 0, 1, 0],
  'ward_number_3': [0, 0, 0, 0, 0],
  'ward_number_4': [0, 0, 0, 0, 1],
  'ward_number_6': [0, 1, 0, 0, 0]}
  )
  tlinear_test_X_1 = pd.DataFrame({'department_BTDT': [1, 1],
  'department_INFO': [0, 0],
  'department_ISD': [0, 0],
  'department_PWDx': [0, 0],
  'reason_Building': [0, 0],
  'reason_Employee & General Comments': [0, 0],
  'reason_Enforcement & Abandoned Vehicles': [1, 1],
  'reason_Generic Noise Disturbance': [0, 0],
  'reason_Housing': [0, 0],
  'reason_Noise Disturbance': [0, 0],
  'reason_Sanitation': [0, 0],
  'reason_Street Cleaning': [0, 0],
  'reason_Street Lights': [0, 0],
  'source_Citizens Connect App': [1, 1],
  'source_City Worker App': [0, 0],
  'source_Constituent Call': [0, 0],
  'subject_Boston Police Department': [0, 0],
  'subject_Inspectional Services': [0, 0],
  "subject_Mayor's 24 Hour Hotline": [0, 0],
  'subject_Public Works Department': [0, 0],
  'subject_Transportation - Traffic Division': [1, 1],
  'ward_number_12': [0, 0],
  'ward_number_14': [0, 0],
  'ward_number_16': [0, 1],
  'ward_number_17': [0, 0],
  'ward_number_19': [0, 0],
  'ward_number_3': [0, 0],
  'ward_number_4': [0, 0],
  'ward_number_6': [1, 0]}
  )
  tlinear_test_X_2 = pd.DataFrame({'department_BTDT': [1, 1, 0, 0],
  'department_INFO': [0, 0, 0, 0],
  'department_ISD': [0, 0, 0, 0],
  'department_PWDx': [0, 0, 1, 1],
  'reason_Building': [0, 0, 0, 0],
  'reason_Employee & General Comments': [0, 0, 0, 0],
  'reason_Enforcement & Abandoned Vehicles': [1, 1, 0, 0],
  'reason_Generic Noise Disturbance': [0, 0, 0, 0],
  'reason_Housing': [0, 0, 0, 0],
  'reason_Noise Disturbance': [0, 0, 0, 0],
  'reason_Sanitation': [0, 0, 0, 0],
  'reason_Street Cleaning': [0, 0, 0, 1],
  'reason_Street Lights': [0, 0, 1, 0],
  'source_Citizens Connect App': [1, 1, 0, 1],
  'source_City Worker App': [0, 0, 1, 0],
  'source_Constituent Call': [0, 0, 0, 0],
  'subject_Boston Police Department': [0, 0, 0, 0],
  'subject_Inspectional Services': [0, 0, 0, 0],
  "subject_Mayor's 24 Hour Hotline": [0, 0, 0, 0],
  'subject_Public Works Department': [0, 0, 1, 1],
  'subject_Transportation - Traffic Division': [1, 1, 0, 0],
  'ward_number_12': [0, 0, 0, 0],
  'ward_number_14': [0, 0, 0, 0],
  'ward_number_16': [0, 1, 0, 0],
  'ward_number_17': [0, 0, 0, 0],
  'ward_number_19': [0, 0, 1, 0],
  'ward_number_3': [0, 0, 0, 0],
  'ward_number_4': [0, 0, 0, 1],
  'ward_number_6': [1, 0, 0, 0]}
  )
  tlogistic_test_y_0 = pd.Series({0: 0, 1: 1, 2: 0, 3: 1, 4: 0, 5: 0, 6: 1, 7: 1, 8: 0, 9: 1}
  )
  tlogistic_test_y_1 = pd.Series({1: 1, 2: 0, 3: 1, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0}
  )
  tlinear_test_y_0 = pd.Series({1: -8759.950555555555,
  3: 47.61472222222222,
  6: 46.986111111111114,
  7: 2195.776388888889,
  9: 2920.1388888888887}
  )
  tlinear_test_y_1 = pd.Series({3: 47.61472222222222, 6: 46.986111111111114}
  )
  tlinear_test_y_2 = pd.Series({3: 47.61472222222222,
  6: 46.986111111111114,
  7: 2195.776388888889,
  9: 2920.1388888888887}
  )

  #call the function with the test data
  logistic_test_X_0, logistic_test_y_0 = clean_and_split_for_logistic(test_data_2022, 0)
  logistic_test_X_1, logistic_test_y_1 = clean_and_split_for_logistic(test_data_2022, 1)

  linear_test_X_0, linear_test_y_0 = clean_and_split_for_linear(test_data_2022, 0)
  linear_test_X_1, linear_test_y_1 = clean_and_split_for_linear(test_data_2022, 1)
  linear_test_X_2, linear_test_y_2 = clean_and_split_for_linear(test_data_2022, 2)


  #check if the function output matches the expected output when reindexed

  test_data = [
      (logistic_test_X_0, tlogistic_test_X_0),
      (logistic_test_X_1, tlogistic_test_X_1),
      (linear_test_X_0, tlinear_test_X_0),
      (linear_test_X_1, tlinear_test_X_1),
      (linear_test_X_2, tlinear_test_X_2),
      (logistic_test_y_0, tlogistic_test_y_0),
      (logistic_test_y_1, tlogistic_test_y_1),
      (linear_test_y_0, tlinear_test_y_0),
      (linear_test_y_1, tlinear_test_y_1),
      (linear_test_y_2, tlinear_test_y_2)
  ]

  for data, expected in test_data:
        if isinstance(data, pd.DataFrame):
            # Sort the DataFrames by index and column names
            data = data.sort_index(axis=0).sort_index(axis=1)
            expected = expected.sort_index(axis=0).sort_index(axis=1)
            # Reset the index to avoid issues with different index types
            data = data.reset_index(drop=True)
            expected = expected.reset_index(drop=True)
            # Compare the DataFrames and assert that they are equal
            #print("Dataframe indices:")
            #print(data.index)
            #print(expected.index)
            #print("Dataframe columns:")
            #print(data.columns)
            #print(expected.columns)
            #diff = data.compare(expected)
            #if not diff.empty:
            #    print(f"DataFrames are different:\n{diff}")
            assert_frame_equal(data, expected, check_dtype=False)
        elif isinstance(data, pd.Series):
            # Sort the Series by index
            #data = data.sort_index(axis=0)
            data = data.rename(None)
            #expected = expected.sort_index(axis=0)
            # Compare the Series and assert that they are equal
            #print("Series indices:")
            #print(data.index)
            #print(expected.index)
            #diff = data.compare(expected)
            #if not diff.empty:
            #    print(f"Series are different:\n{diff}")
            assert_series_equal(data, expected, check_dtype=False)



In [119]:
test_data_clean_functions()

##Ingest all the data 

Here is the link to all the data sets:

https://data.boston.gov/dataset/311-service-requests

In [33]:

# Get a list of all CSV files in the directory
all_files = [url_2023, url_2022, url_2021, url_2020, url_2019, url_2018, url_2017, url_2016, url_2015, url_2014, url_2013, url_2012, url_2011]

# Create an empty list to store the dataframes
dfs = []

# Loop through the files and load them into dataframes
for file in all_files:
  df = pd.read_csv(file)
  dfs.append(df)

<ipython-input-33-c7e6c2df8ad4>:9: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
<ipython-input-33-c7e6c2df8ad4>:9: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [120]:
#check that the files all have the same number of columns, and the same names
for i in range(len(dfs)):
  if dfs[i].shape[1] != dfs[0].shape[1]:
    print('Error: File', i, 'does not have the same number of columns as File 0')
  else:
    print('File', i, 'has same number of columns as File 0')
  if not dfs[i].columns.equals(dfs[0].columns):
    print('Error: File', i, 'does not have the same column names and order as File 0')
  else:
    print('File', i, 'has the same column name and order as File 0')

File 0 has same number of columns as File 0
File 0 has the same column name and order as File 0
File 1 has same number of columns as File 0
File 1 has the same column name and order as File 0
File 2 has same number of columns as File 0
File 2 has the same column name and order as File 0
File 3 has same number of columns as File 0
File 3 has the same column name and order as File 0
File 4 has same number of columns as File 0
File 4 has the same column name and order as File 0
File 5 has same number of columns as File 0
File 5 has the same column name and order as File 0
File 6 has same number of columns as File 0
File 6 has the same column name and order as File 0
File 7 has same number of columns as File 0
File 7 has the same column name and order as File 0
File 8 has same number of columns as File 0
File 8 has the same column name and order as File 0
File 9 has same number of columns as File 0
File 9 has the same column name and order as File 0
File 10 has same number of columns as Fi

In [121]:
# Concatenate the dataframes into a single dataframe
df_all = pd.concat(dfs, ignore_index=True)

In [122]:
#save ram by deleting the dfs variable
del dfs

In [123]:
df_all['type'].nunique()

212

In [125]:
data = df_all.copy()
data['open_dt'] = pd.to_datetime(data['open_dt'])
data['closed_dt'] = pd.to_datetime(data['closed_dt'])

In [148]:
data.loc[data['closed_dt'] < data['open_dt']]['source'].value_counts()

Employee Generated    437
City Worker App       353
Constituent Call        1
Name: source, dtype: int64

In [ ]:
logistic_X, logistic_y = clean_and_split_for_logistic(df_all, 1)

In [ ]:
linear_X, linear_y = clean_and_split_for_linear(df_all, 2)

In [ ]:
linear_X.describe()



,subject_Animal Control,subject_Boston Police Department,subject_Boston Water & Sewer Commission,subject_CRM Application,subject_Consumer Affairs & Licensing,subject_Disability Department,subject_Inspectional Services,subject_Mayor's 24 Hour Hotline,subject_Neighborhood Services,subject_Parks & Recreation Department,...,ward_number_20,ward_number_21,ward_number_22,ward_number_3,ward_number_4,ward_number_5,ward_number_6,ward_number_7,ward_number_8,ward_number_9
count,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,...,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06,2.325113e+06
mean,5.866812e-03,1.621427e-04,3.971850e-03,4.300866e-07,3.440693e-06,9.083429e-04,7.542085e-02,3.842265e-02,4.859979e-05,5.733227e-02,...,5.820921e-02,4.030600e-02,4.248224e-02,8.972897e-02,4.273599e-02,8.104767e-02,5.123837e-02,4.257384e-02,3.411490e-02,3.039981e-02
std,7.637012e-02,1.273250e-02,6.289735e-02,6.558099e-04,1.854908e-03,3.012504e-02,2.640693e-01,1.922144e-01,6.971187e-03,2.324765e-01,...,2.341387e-01,1.966760e-01,2.016867e-01,2.857931e-01,2.022613e-01,2.729084e-01,2.204836e-01,2.018944e-01,1.815243e-01,1.716848e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
linear_y.describe()

count    2.325113e+06
mean     3.958667e+02
std      1.789195e+03
min      0.000000e+00
25%      1.416944e+00
50%      1.710167e+01
75%      1.176658e+02
max      6.989429e+04
Name: survival_time_hours, dtype: float64

In [ ]:
#Train a logistic regression model

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(logistic_X, logistic_y, test_size=0.2, random_state=42)

# Build model
model_logistic = keras.Sequential([
    keras.layers.Dense(units=1, input_shape=(X_train.shape[1],), activation='sigmoid')
])

# Compile model
model_logistic.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model_logistic.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate model
test_loss, test_acc = model_logistic.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

Epoch 1/10
64032/64032 [==============================] - 111s 2ms/step - loss: 0.3003 - accuracy: 0.8633
Epoch 2/10
64032/64032 [==============================] - 123s 2ms/step - loss: 0.2958 - accuracy: 0.8654
Epoch 3/10
64032/64032 [==============================] - 109s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 4/10
64032/64032 [==============================] - 107s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 5/10
64032/64032 [==============================] - 112s 2ms/step - loss: 0.2958 - accuracy: 0.8655
Epoch 6/10
64032/64032 [==============================] - 107s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 7/10
64032/64032 [==============================] - 113s 2ms/step - loss: 0.2958 - accuracy: 0.8655
Epoch 8/10
64032/64032 [==============================] - 109s 2ms/step - loss: 0.2958 - accuracy: 0.8657
Epoch 9/10
64032/64032 [==============================] - 122s 2ms/step - loss: 0.2958 - accuracy: 0.8655
Epoch 10/10
16008/16008 [=====================

Let's delete the df_all variable to save RAM before we train our linear regression model

In [ ]:
del df_all

In [ ]:
#Train a linear regression model

start_time = datetime.now()
print("Starting Training at {}".format(start_time))

scaler = StandardScaler()
X_scaled = scaler.fit_transform(linear_X) # scale the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, linear_y, test_size=0.2, random_state=42)

# split the data again to create a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# define the model architecture
model_linear = keras.Sequential([
    keras.layers.Dense(units=1, input_dim=X_train.shape[1])
])

# compile the model
model_linear.compile(optimizer='adam', loss='mean_squared_error')

# train the model
model_linear.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_val, y_val))

end_time = datetime.now()
total_time = (end_time - start_time)
print("Ending Training at {}".format(end_time))
print("Training took {}".format(total_time))

Starting Training at 2023-05-09 16:59:26.027579
Epoch 1/50
46503/46503 [==============================] - 108s 2ms/step - loss: 3270872.0000 - val_loss: 3272695.5000
Epoch 2/50
46503/46503 [==============================] - 84s 2ms/step - loss: 3174310.5000 - val_loss: 3191508.0000
Epoch 3/50
46503/46503 [==============================] - 84s 2ms/step - loss: 3110366.7500 - val_loss: 3141374.2500
Epoch 4/50
46503/46503 [==============================] - 93s 2ms/step - loss: 3103389.2500 - val_loss: 3111857.0000
Epoch 5/50
46503/46503 [==============================] - 92s 2ms/step - loss: 3087338.5000 - val_loss: 3088312.0000
Epoch 6/50
46503/46503 [==============================] - 91s 2ms/step - loss: 3086873.2500 - val_loss: 3079226.2500
Epoch 7/50
46503/46503 [==============================] - 84s 2ms/step - loss: 3100280.0000 - val_loss: 3069662.7500
Epoch 8/50
46503/46503 [==============================] - 83s 2ms/step - loss: 3102633.7500 - val_loss: 3059529.5000
Epoch 9/50
4650

In [ ]:
model_linear.save("model_linear.h5")

In [ ]:
model_logistic.save("model_logistic.h5")

We save these models so that we can start building a website and API to make these models available to the public